## Helpful Python Libraries

In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib as mp
import matplotlib.pyplot as plt
from numpy.lib import recfunctions
import datetime
import numpy.lib.recfunctions as recfn
import boto3
import sys
from IPython.display import clear_output
from scipy import stats, fftpack
from scipy.stats import kurtosis, skew, iqr, t
import pickle
from scipy import io as sio
s3 = boto3.resource('s3')
from io import BytesIO

# Introduction

In this exercise we look at the aircraft take offs telemetry data to identify anomalies in a bid to identify any abnormality. While commercial flying safety standards are high and well regulated; Take offs can be challenging and engine stalling can be challenge.

Here we shall use limited data wrangle on the existing NASA Dashlink Telemetry data to build a  representative feature set of time varying sensor signals

## Constants

In [6]:
FLGHTLEN = .75 # Filter out anything that is less than this flight time
IN_BKT = 'iia-vault-telemetry-practice-unzipped'
SMPL_SIZES= [.01,.01,.1,.1]
RANDSEED= 42 # Urban Data Science Myth- Apparently gives best results!!
# NUMOFVARS= 30 # Number of variables from the mat files to be selected
PRINTATEVERY= 2500 # Key Values

# Base Variables

There are 186 data variables in the Telemetry data set. What is relevant to aircraft take offs?

***To Do:***

* Familiarize yourself with the definition of the following variables
* Look at the wiki page link <a href="https://wiki.iiaweb.com/index.php?title=VAULT/RawData/DASHLink#Format" target="_blank"> here</a>, look at the description and identify a few variables that may be meaningful for feature creation

In [2]:
continuous = ['BLAC','CTAC','WS','ALTR']
degs = ['AOA1','AOA2','PTCH','ROLL']

## Bucket Read

This is a old function with bit of a twist. Sorts the keys by date time in ascending order

In [7]:
def s3_bucket_object_keys(bucket_name= IN_BKT):
    bucket = s3.Bucket(bucket_name)
    key_list=[]
    for key in bucket.objects.all():
        key_list.append(key)
    sorted(key_list, key = lambda x: int(x.key[16:28]))
    return(key_list)

In [8]:
key_list = s3_bucket_object_keys()

## Sample Prep

Picks up a set of non repeating samples as defined by the SMPL_SIZES list

In [9]:
def pick_random_sets(mat_list, pct_size=SMPL_SIZES):
    np.random.seed(seed=RANDSEED)
    smpl_sets=[]
    for pct in pct_size:
        sz = int(pct*len(mat_list))
        smpl_sets.append(np.random.choice(mat_list, size=sz, replace=False))
    return(smpl_sets)

In [10]:
samples= pick_random_sets(key_list)

In [13]:
def load_mat_file(s3_key, bucketname= IN_BKT):
    s3 = boto3.resource('s3')
    obj = s3.Object(bucketname, s3_key.key)
    inMATFile = obj.get()['Body'].read()
    raw_mat = sio.loadmat(BytesIO(inMATFile))
    return(raw_mat)

# Flight Length 

Some of the flights are relatively small and may even be trial runs. In order to calculate the length of flight we need to measure length of one of the 186 variable in the mat file with 1 Hz sampling frequency

***To Do:***
What would be a good candidate variable to use in place of 'SAT'

In [14]:
def flight_len(mat):
    return(round(len(mat['SAT']['data'][0][0])/3600,2))

In [15]:
def flght_start_end(mat):
    lat = mat['LATP']['data'][0][0]
    long = mat['LONP']['data'][0][0]
    return(lat[5][0],long[5][0], lat[-5][0],long[-5][0])

# Filter by flight length

This function looks for flights with a specific FLGHTLEN and valid landing gear keys

In [16]:
def flght_filter(mat, flight_length= FLGHTLEN):
    t_len= len(mat['LGDN']['data'][0][0])/3600
    valid_lnd= len(np.unique(mat['LGDN']['data'][0][0], return_counts=False))
    if t_len >= flight_length and valid_lnd >= 2:
        return(True)
    else:
        return(False)

## Indexes for landing gear up and down

In [17]:
def steady_flight(mat):
    lndg= mat['LGUP']['data'][0][0]
    lndg_flg_chg= []
    for i in range(len(lndg)-1):
        if (lndg[i]!=lndg[i+1]):
            lndg_flg_chg.append(i+1)
    return(lndg_flg_chg)

## GESD Custom formula

Generalized Extreme Studentized Deviate Test

In GESD anomalies are progressively evaluated removing the worst offenders and recalculating the test statistics and critical values, or more simply you can say that a range is recalculated after identifying the anomalies in an iterative way.

Twitters Anamolize package

In [18]:
from scipy.stats import kurtosis, skew, iqr, t
def grubbs_stat(y):
    std_dev = np.std(y)
    avg_y = np.mean(y)
    abs_val_minus_avg = abs(y - avg_y)
    max_of_deviations = max(abs_val_minus_avg)
    max_ind = np.argmax(abs_val_minus_avg)
    Gcal = max_of_deviations/ std_dev
#     print("Grubbs Statistics Value : {}".format(Gcal))
    return(Gcal, max_ind)

In [19]:
from scipy.stats import kurtosis, skew, iqr, t
def calculate_critical_value(size, alpha):
    t_dist = t.ppf(1 - alpha / (2 * size), size - 2)
    numerator = (size - 1) * np.sqrt(np.square(t_dist))
    denominator = np.sqrt(size) * np.sqrt(size - 2 + np.square(t_dist))
    critical_value = numerator / denominator
#     print("Grubbs Critical Value: {}".format(critical_value))
    return(critical_value)

In [20]:
from scipy.stats import kurtosis, skew, iqr, t
def check_G_values(Gs, Gc, inp, max_index, eco= False):
    if eco == True:
        if Gs > Gc:
            print('{} is an outlier. G > G-critical: {:.4f} > {:.4f} \n'.format(inp[max_index], Gs, Gc))
        else:
            print('{} is not an outlier. G > G-critical: {:.4f} < {:.4f} \n'.format(inp[max_index], Gs, Gc))

In [21]:
from scipy.stats import kurtosis, skew, iqr, t
def ESD_Test(input_series, alpha, max_outliers):
    for iterations in range(max_outliers):
        Gcritical = calculate_critical_value(len(input_series), alpha)
        Gstat, max_index = grubbs_stat(input_series)
        check_G_values(Gstat, Gcritical, input_series, max_index)
        input_series = np.delete(input_series, max_index)
        return([Gcritical, Gstat, max_index])

# Continuous feature

## Time aggregate features

***To Do:***

Can you think of any other metric that might be of interest to represent a time series?
Can we take a stab at one

In [23]:
from scipy.stats import kurtosis, skew, iqr, t
def conti_stats(mat, continuous, lngd_idx):
    var_stats={}
    for v in continuous:
#         var_stats['key']= v
        stats= {}
        data =  mat[v]['data'][0][0]
        rate = mat[v]['Rate'][0][0][0][0]
        data = np.array(data[0: int(lngd_idx[0]*rate): rate])
        stats['mean']= np.mean(data)
        stats['uniques']= len(np.unique(data))
        stats['variance']= np.var(data)
        stats['kurt']= kurtosis(data)[0]
        stats['skew']= skew(data)[0]
        stats['iqr']=iqr(data)
        try:
            stats['GESD']= ESD_Test(np.squeeze(data).T,0.05,5)
        except: 
            stats['GESD']= [np.NaN, np.NaN, np.NaN]
        var_stats[v]= stats
    return(var_stats)

## Degrees Features

Degrees are angular measures- neither interval or ratios. One usual way of handling is to convert them into radian and then convert them into SIN/ COS which are obvious ratios


In [24]:
def fromdeg(d):
    r = d * np.pi / 180.
    return(np.cos(r), np.sin(r))

In [25]:
def deg_stats(mat, degs, lngd_idx):
    var_stats= {}
    for d in degs:
        data =  mat[d]['data'][0][0]
        rate = mat[d]['Rate'][0][0][0][0]
        data = np.array(data[0: int(lngd_idx[0]*rate): rate])
        [c,s]= fromdeg(data)
        for i, data in enumerate([c,s]):
            if i == 0:
                var_name = d+'_'+'cos'
            else:
                var_name = d+'_'+'sin'
            stats={}
            stats['mean']= np.mean(data)
            stats['uniques']= len(np.unique(data))
            stats['variance']= np.var(data)
            stats['kurt']= kurtosis(data)[0]
            stats['skew']= skew(data)[0]
            stats['iqr']=iqr(data)
            try:
                stats['GESD']= ESD_Test(np.squeeze(data).T,0.05,5)
            except: 
                stats['GESD']= [np.NaN, np.NaN, np.NaN]
            var_stats[var_name]= stats
    return(var_stats) 

## Feature Creation Function 

Loops through all keys, open mat files, extracts data and build the features- One flight at a time

In [26]:
def create_sample_dictionary(sample_set_num=0):
    flghts =0
    start = datetime.datetime.now()
    print('Main start:', datetime.datetime.now())
    mat_list = s3_bucket_object_keys()
    print('Key List Read:', datetime.datetime.now())
    smpl_set= pick_random_sets(mat_list)
    print('Key List Split:', datetime.datetime.now())
    s3_keys= samples[sample_set_num]
    takeoff_stats= {}
    for ind, key in enumerate(s3_keys):
        flght_detail= key.key[:28]
        train = load_mat_file(key)
        stats={}
        if flght_filter(train)== True:
            take_off_idx= steady_flight(train)
#             var_list= select_variables(train, [0,take_off_idx[0]])
            flghts +=1
            try:
                stats['conti']= conti_stats(train, continuous, take_off_idx)
                stats['degs']= deg_stats(train, degs, take_off_idx)
            except:
                continue
            takeoff_stats[flght_detail]= stats
            if ind% PRINTATEVERY == 0:
                print('Processing Key:', key.key, datetime.datetime.now())
#                 print(flght_detail, stats)
    end = datetime.datetime.now()
    print("Time Taken to run the function ",str(end-start))
    print("Total Flights that meet criteria:", flghts)
    return(takeoff_stats)

In [27]:
takeoff_stats= create_sample_dictionary()

Main start: 2020-09-01 10:26:16.113388
Key List Read: 2020-09-01 10:26:37.884483
Key List Split: 2020-09-01 10:26:38.323545
Processing Key: Flight 659/3/659200201270513.mat 2020-09-01 10:26:38.570340


<ipython-input-18-fd0cb43dfec9>:8: RuntimeWarning: invalid value encountered in double_scalars
  Gcal = max_of_deviations/ std_dev


Time Taken to run the function  0:03:02.711849
Total Flights that meet criteria: 572


# Feature Management

Feature needs to be created in a repeatable and reproducable manner. More over real world problems tend to have a very large feature set and can be tedious to refresh and hence decisions needs to be taken about the freshness of feature which drive the models

In [28]:
file_to_write = open("flight_stats_tutorial.pickle", "wb")
pickle.dump(takeoff_stats, file_to_write)

In [29]:
with open('flight_stats_tutorial.pickle', 'rb') as f:
    x = pickle.load(f)

In [35]:
i=0
for key, value in x.items():
    i +=1
    if i%500==0:
        print(key)
        print(value)

Flight 655/1/655200104251428
{'conti': {'BLAC': {'mean': 0.0042056626175776156, 'uniques': 150, 'variance': 0.0039029210911010284, 'kurt': 8.72461008887516, 'skew': 2.9877432060346774, 'iqr': 0.01660909021803736, 'GESD': [3.9740915882336183, 4.28022956655909, 750]}, 'CTAC': {'mean': -0.00046687632985478847, 'uniques': 57, 'variance': 4.07435156053945e-05, 'kurt': 20.073872164146135, 'skew': -0.031759991144461514, 'iqr': 0.0, 'GESD': [3.9740915882336183, 7.87926749321862, 730]}, 'WS': {'mean': 0.09727480293562014, 'uniques': 6, 'variance': 0.299309185786443, 'kurt': 41.096444799849, 'skew': 6.274601534313071, 'iqr': 0.0, 'GESD': [3.9740915882336183, 8.925735964448865, 738]}, 'ALTR': {'mean': 25.770833333333332, 'uniques': 37, 'variance': 51540.1974826389, 'kurt': 45.28403417425217, 'skew': 6.593205581097803, 'iqr': 48.0, 'GESD': [3.9740915882336183, 9.259915383155764, 764]}}, 'degs': {'AOA1_cos': {'mean': 0.9989343731167027, 'uniques': 127, 'variance': 2.2435556032831233e-06, 'kurt': 28